In [ ]:
import pandas as pd


from __future__ import print_function
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from bs4 import BeautifulSoup
import re
import keras.regularizers
from keras.callbacks import ModelCheckpoint


"""
pd.to_pickle(obj = text_neg,path="/home/abhinav/data/saved_samples")
pd.to_pickle(obj = text_pos,path = "/home/abhinav/data/saved_postives")
pd.to_pickle(obj = embeddings_index,path = "/home/abhinav/data/first_model/embeddings_index_dict")
pd.to_pickle(obj = tokenizer, path = "/home/abhinav/data/first_model/tokenizer")
"""

text_neg = pd.read_pickle("/home/abhinav/data/saved_samples")
text_pos = pd.read_pickle("/home/abhinav/data/saved_postives")



In [2]:
BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
VALIDATION_SPLIT = 0.3

In [11]:
print('Indexing word vectors.')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
#embeddings_index = pd.read_pickle("/home/abhinav/data/first_model/embeddings_index_dict")

Indexing word vectors.


In [12]:
pd.to_pickle(embeddings_index,"/home/abhinav/data/fourth_model/embedding_hash_200d")

In [3]:
embeddings_index = pd.read_pickle("/home/abhinav/data/fourth_model/embedding_hash_200d")

In [4]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
a = np.arange(151402)
np.random.seed(0)
indices_sample = np.random.permutation(a)
np.random.shuffle(indices_sample)

b = np.arange(1645)
np.random.seed(10)
indices_sample_protest = np.random.permutation(b)
np.random.shuffle(indices_sample_protest)

test_sample_indices_protest = indices_sample_protest[:165]
val_sample_indices_protest = indices_sample_protest[165:495]
train_sample_indices_protest = indices_sample_protest[495:]

test_sample_indices = indices_sample[:15000]
val_sample_indices = indices_sample[15000:45000]
train_sample_indices = indices_sample[45000:]

In [5]:
text_neg_val = []
text_neg_train = []
text_neg_test = []
count = 0
count1 = 0
for name in sorted(os.listdir(TEXT_DATA_DIR+"/not_protest")):
    path = os.path.join(TEXT_DATA_DIR+'/not_protest', name)
    f = open(path)
    t = f.read()
    t = t.split("</DOC>")[:-1]
    for i,docs in enumerate(t):
        t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
        if (count%10000)==0:
            print (count,": done")
        if count in test_sample_indices:
            text_neg_test.append(t[i])
        elif count in val_sample_indices:
            text_neg_val.append(t[i])
        elif count in train_sample_indices:
            text_neg_train.append(t[i])
        count=count+1
    f.close()
print(count)

0 : done
10000 : done
20000 : done
30000 : done
40000 : done
50000 : done
60000 : done
70000 : done
80000 : done
90000 : done
100000 : done
110000 : done
120000 : done
130000 : done
140000 : done
150000 : done
151402


In [6]:
text_pos_val = []
text_pos_train  = []
text_pos_test = []
count = 0
for name in sorted(os.listdir(TEXT_DATA_DIR+"/protest")):
    path = os.path.join(TEXT_DATA_DIR+'/protest', name)
    f = open(path)
    t = f.read()
    t = t.split("</DOC>")[:-1]
    for i,docs in enumerate(t):
        t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
        if count in test_sample_indices_protest:
            text_pos_test.append(t[i])
        elif count in val_sample_indices_protest:
            text_pos_val.append(t[i])
        elif count in train_sample_indices_protest:
            text_pos_train.append(t[i])
        count= count +1
    f.close()
print(count)

1645


In [9]:
text_data  =  text_neg_train + text_pos_train + text_neg_val + text_pos_val

label_neg_train = np.zeros((len(text_neg_train),),dtype=int)
label_pos_train = np.ones((len(text_pos_train),),dtype=int)
labels_train = np.append(label_neg_train,label_pos_train)

label_neg_val = np.zeros((len(text_neg_val),),dtype=int)
label_pos_val = np.ones((len(text_pos_val),),dtype=int)
labels_val = np.append(label_neg_val,label_pos_val)

from keras.preprocessing.text import Tokenizer,pad_sequences
from keras.utils import to_categorical

MAX_NB_WORDS =50000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)

x_train = data[:len(text_neg_train)+len(text_pos_train)]
x_val = data[len(text_neg_train)+len(text_pos_train):]

y_train = to_categorical(np.asarray(labels_train))
y_val = to_categorical(np.asarray(labels_val))

test_seq = tokenizer.texts_to_sequences(text_pos_test)
x_test_pos = pad_sequences(test_seq,maxlen = MAX_SEQUENCE_LENGTH)

y_test_positives = np.ones((len(text_pos_test),))
temp = np.append(np.zeros((1,)),y_test_positives)
y_test_positives = to_categorical(temp)[1:]

test_seq_ = tokenizer.texts_to_sequences(text_neg_test)
x_test_neg = pad_sequences(test_seq_,maxlen = MAX_SEQUENCE_LENGTH)

y_test_negatives = np.zeros((len(text_neg_test),))
temp = np.append(np.ones((1,)),y_test_negatives)
y_test_negatives = to_categorical(temp)[1:]

x_test = np.append(x_test_neg,x_test_pos,axis=0)
y_test = to_categorical(np.append(np.zeros((len(text_neg_test),)),np.ones((len(text_pos_test),))))
pd.to_pickle(obj=[x_test,y_test], path=fourth_model_dir + "test_dataset")

x_train_neg = data[:106402]

np.random.seed(0)
np.random.shuffle(x_train_neg)

x_train_neg_10 = x_train_neg[:12000]
x_train_neg_1 = x_train_neg[:1200]

x_train_pos = data[106402:106402+1150]

x_train = np.append(x_train_neg_10,x_train_pos,axis = 0)
y_train = to_categorical(np.append(np.zeros(12000,),np.ones(1150,),axis = 0))

tokenizer = pd.read_pickle("/home/abhinav/data/fourth_model/tokenizer")

In [2]:
data = pd.read_pickle("/home/abhinav/data/fourth_model/data_in_sequence_form")

In [15]:
pd.to_pickle(obj=tokenizer,path = "/home/abhinav/data/fourth_model/tokenizer")
pd.to_pickle(obj=data, path = "/home/abhinav/data/fourth_model/data_in_sequence_form")

In [31]:
fourth_model_dir = "/home/abhinav/data/fourth_model/"

In [32]:
pd.to_pickle(obj=[x_val, y_val], path= fourth_model_dir + "validation_dataset")

to convert downsample the non protest data:

In [4]:
x_train_neg = data[:106402]



np.random.seed(0)
np.random.shuffle(x_train_neg)


x_train_neg_10 = x_train_neg[:12000]
x_train_neg_1 = x_train_neg[:1200]



x_train_pos = data[106402:106402+1150]

x_train = np.append(x_train_neg_10,x_train_pos,axis = 0)
y_train = to_categorical(np.append(np.zeros(12000,),np.ones(1150,),axis = 0))

In [10]:
pd.to_pickle([x_train_pos,y_train_pos],"/home/abhinav/data/second_model/train_positive_data")

to create the traininig data after downsampling

In [6]:
pd.to_pickle([x_train,y_train],"/home/abhinav/data/fourth_model/train_data_10ratio")

pd.to_pickle(obj=[x_train,y_train],path=second_model_dir+"training_dataset")

to get only the protest examples in the validation dataset

In [54]:
x_val_positives = data[-330:]
y_val_positives = np.ones((330,))
temp = np.append(np.zeros((1,)),y_val_positives)
y_val_positives = to_categorical(temp)[1:]

In [5]:
pd.to_pickle([x_val_positives,y_val_positives],path = "/home/abhinav/data/second_model/val_positive_data")

to get only the non protest examples of the validation dataset

In [56]:
x_val_negatives = data[-30330:-330]
y_val_negatives = np.zeros((30000,))
temp_ = np.append(np.ones((1,)),y_val_negatives)
y_val_negatives = to_categorical(temp_)[1:]

In [ ]:
pd.to_pickle(obj = [x_val_negatives,y_val_negatives] , path="/home/abhinav/data/fourth_model/val_negative_data")

to get the positives & negatives from the test dataset

In [19]:
test_seq = tokenizer.texts_to_sequences(text_pos_test)
x_test_pos = pad_sequences(test_seq,maxlen = MAX_SEQUENCE_LENGTH)

y_test_positives = np.ones((len(text_pos_test),))
temp = np.append(np.zeros((1,)),y_test_positives)
y_test_positives = to_categorical(temp)[1:]

test_seq_ = tokenizer.texts_to_sequences(text_neg_test)
x_test_neg = pad_sequences(test_seq_,maxlen = MAX_SEQUENCE_LENGTH)

y_test_negatives = np.zeros((len(text_neg_test),))
temp = np.append(np.ones((1,)),y_test_negatives)
y_test_negatives = to_categorical(temp)[1:]

x_test = np.append(x_test_neg,x_test_pos,axis=0)
y_test = to_categorical(np.append(np.zeros((len(text_neg_test),)),np.ones((len(text_pos_test),))))
pd.to_pickle(obj=[x_test,y_test], path=fourth_model_dir + "test_dataset")

In [34]:

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Preparing embedding matrix.


pd.to_pickle(embedding_matrix, fourth_model_dir + "embedding_matrix_for_selected_words")

In [51]:
indice_shuffler = np.arange(x_train.shape[0])
np.random.seed(0)
np.random.shuffle(indice_shuffler)
x_train = x_train[indice_shuffler]
y_train = y_train[indice_shuffler]

In [52]:
x_train.shape

(13150, 1000)

In [37]:
indice_shuffler_ = np.arange(x_val.shape[0])
np.random.seed(0)
np.random.shuffle(indice_shuffler_)
x_val = x_val[indice_shuffler_]
y_val = y_val[indice_shuffler_]

pretrained = False
if pretrained:
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,

In [38]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Dropout

In [60]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = to_categorical(self.model.predict_classes(self.validation_data[0],batch_size = 512))
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("Weighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

In [80]:
model_ = Sequential()
model_.add(Embedding(num_words,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH))
model_.add(Conv1D(128, 5, activation='relu'))
model_.add(MaxPooling1D(5))
model_.add(Conv1D(128, 5, activation='relu'))
model_.add(MaxPooling1D(5))
model_.add(Conv1D(128, 5, activation='relu'))
model_.add(MaxPooling1D(35))
model_.add(Flatten())
model_.add(Dense(128,activation="relu"))
model_.add(Dense(2,activation="softmax"))

In [82]:
model_.compile(loss = "binary_crossentropy",metrics = ["acc"], 
                       optimizer=ndm)

In [83]:
initial_model_weights = model_.get_weights()

In [64]:
model_.set_weights(initial_model_weights)

In [84]:
metrics = Metrics(fourth_model_dir + "none")
model_.fit(x_train,y_train
                  ,batch_size = 128,
                  epochs = 4,
                  class_weight = {0. :1, 1. : 7},
                  validation_data = [x_test,y_test],
                   callbacks = [metrics]
                  )

Train on 13150 samples, validate on 15165 samples
Epoch 1/4
15165/15165 [==============================] - 91s    
Weighted F1 score found on Validation dataset :  (array([ 0.99252811,  0.09705373]), array([ 0.96526667,  0.33939394]), array([ 0.97870758,  0.1509434 ]), array([15000,   165]))
13150/13150 [==============================] - 407s - loss: 0.9513 - acc: 0.8644 - val_loss: 0.1606 - val_acc: 0.9585
Epoch 2/4
15165/15165 [==============================] - 91s    
Weighted F1 score found on Validation dataset :  (array([ 0.9961544 ,  0.08098337]), array([ 0.91526667,  0.67878788]), array([ 0.95399903,  0.14470284]), array([15000,   165]))
13150/13150 [==============================] - 403s - loss: 0.5391 - acc: 0.8649 - val_loss: 0.2138 - val_acc: 0.9127
Epoch 3/4
15165/15165 [==============================] - 86s    
Weighted F1 score found on Validation dataset :  (array([ 0.99643273,  0.05242291]), array([ 0.8566    ,  0.72121212]), array([ 0.92124037,  0.09774127]), array([1

In [85]:
model_.set_weights(initial_model_weights)

In [86]:
metrics = Metrics(fourth_model_dir + "none")
model_.fit(x_train,y_train
                  ,batch_size = 128,
                  epochs = 4,
                  class_weight = {0. :1, 1. : 8},
                  validation_data = [x_test,y_test],
                   callbacks = [metrics]
                  )

Train on 13150 samples, validate on 15165 samples
Epoch 1/4
15165/15165 [==============================] - 86s    
Weighted F1 score found on Validation dataset :  (array([ 0.99818182,  0.0178634 ]), array([ 0.4392    ,  0.92727273]), array([ 0.61      ,  0.03505155]), array([15000,   165]))
13150/13150 [==============================] - 389s - loss: 1.0563 - acc: 0.8143 - val_loss: 0.7295 - val_acc: 0.4445
Epoch 2/4
15165/15165 [==============================] - 87s    
Weighted F1 score found on Validation dataset :  (array([ 0.9984148 ,  0.03858268]), array([ 0.7558    ,  0.89090909]), array([ 0.86033011,  0.07396226]), array([15000,   165]))
13150/13150 [==============================] - 389s - loss: 0.5849 - acc: 0.8281 - val_loss: 0.5090 - val_acc: 0.7573
Epoch 3/4
15165/15165 [==============================] - 82s    
Weighted F1 score found on Validation dataset :  (array([ 0.99557869,  0.09019264]), array([ 0.93073333,  0.62424242]), array([ 0.96206457,  0.15761285]), array([1

In [87]:
model_.set_weights(initial_model_weights)

In [88]:
model_.fit(x_train,y_train
                  ,batch_size = 128,
                  epochs =4,
                  class_weight = {0. :1, 1. : 10},
                  validation_data = [x_val,y_val],
                   callbacks = [metrics]
                  )

Train on 13150 samples, validate on 30330 samples
Epoch 1/4
30330/30330 [==============================] - 173s   
Weighted F1 score found on Validation dataset :  (array([ 0.9945685 ,  0.04871661]), array([ 0.87893333,  0.56363636]), array([ 0.93318233,  0.08968177]), array([30000,   330]))
13150/13150 [==============================] - 558s - loss: 1.1628 - acc: 0.6973 - val_loss: 0.3204 - val_acc: 0.8755
Epoch 2/4
30330/30330 [==============================] - 172s   
Weighted F1 score found on Validation dataset :  (array([ 0.99604775,  0.04374172]), array([ 0.83166667,  0.7       ]), array([ 0.90646515,  0.08233826]), array([30000,   330]))
13150/13150 [==============================] - 557s - loss: 0.6770 - acc: 0.8260 - val_loss: 0.3724 - val_acc: 0.8302
Epoch 3/4
30330/30330 [==============================] - 170s   
Weighted F1 score found on Validation dataset :  (array([ 0.99465024,  0.05734656]), array([ 0.89863333,  0.56060606]), array([ 0.94420706,  0.10404949]), array([3

In [ ]:
from keras.models import load_model
model1 = load_model("/home/abhinav/data/second_model/the_best_model_with_trained_word_embeddings")

In [ ]:
model_.fit(x_train,y_train
                  ,batch_size = 128,
                  epochs =2,
                  class_weight = {0. :1, 1. : 10},
                  validation_data = [x_val,y_val],
                   callbacks = [metrics]
                  )

Train on 13150 samples, validate on 30330 samples
Epoch 1/2
 3072/30330 [==>...........................] - ETA: 294s loss: 0.0314 - a

In [57]:
model_.evaluate(x_val_negatives,y_val_negatives,batch_size=512)

30000/30000 [==============================] - 173s   


[1.7458493810017903, 0.11363333338101705]

In [58]:
model_.evaluate(x_test_pos,y_test_positives,batch_size=512)

165/165 [==============================] - 0s


[0.072426959872245789, 1.0]

In [59]:
model_.evaluate(x_test_neg,y_test_negatives,batch_size=512)

15000/15000 [==============================] - 85s    


[1.7384219600677491, 0.11386666659514109]

In [412]:
model_1 = load_model(second_model_dir+"lets_try_again_to_save")

In [413]:
model_1.evaluate(x_test_pos,y_test_positives,batch_size=512)

175/175 [==============================] - 1s


[0.85618859529495239, 0.691428542137146]

In [414]:
model_1.evaluate(x_val_positives,y_val_positives)

350/350 [==============================] - 1s     


[0.88645219734736846, 0.69714285884584704]

In [416]:
model_1.evaluate(x_val_negatives,y_val_negatives,batch_size=512)

30000/30000 [==============================] - 105s   


[0.15647533753712972, 0.93570000009536747]

In [404]:
to_categorical(model_.predict_classes(x_news))

2/2 [==============================] - 0s


array([[ 0.,  1.],
       [ 0.,  1.]])

In [366]:
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',metrics=["acc"]
              ,optimizer=ndm)

In [ ]:
initial_weights = model.get_weights()

In [362]:
model.set_weights(initial_weights)

In [367]:
metrics = Metrics(second_model_dir + "trying_to_save_model")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=2,
          class_weight = { 0. : 1, 1. : 8 },
          validation_data=(x_val, y_val),
          callbacks =[metrics]
         )

Train on 11228 samples, validate on 30350 samples
Epoch 1/2
11136/11228 [============================>.] - ETA: 1s - loss: 1.2254 - acc: 0.6608Epoch 00000: improved f1 to 0.98274, saving model to /home/abhinav/data/second_model/trying_to_save_model
Weighted F1 score found on Validation dataset :  0.982735252488
11228/11228 [==============================] - 334s - loss: 1.2240 - acc: 0.6630 - val_loss: 0.6886 - val_acc: 0.9885
Epoch 2/2
11228/11228 [==============================] - 323s - loss: 1.2239 - acc: 0.8906 - val_loss: 0.6867 - val_acc: 0.9885


In [372]:
metrics.f1s

0.98273525248788973

In [371]:
evaluate(x_val,y_val,model)

0.98273525248788973

In [298]:
second_model_dir = "/home/abhinav/data/second_model/"

In [ ]:
model1 = load_model(second_model_dir + "saved_model_classifier")

In [301]:
print (model1.evaluate(x_val_positives,y_val_positives,batch_size=512))
print (model1.evaluate(x_val_negatives,y_val_negatives,batch_size=512))

350/350 [==============================] - 1s
[0.57591825723648071, 0.96285712718963623]
30000/30000 [==============================] - 102s   
[0.72436426213582361, 0.32140000011126202]


In [368]:
model.evaluate(x_val_positives,y_val_positives,batch_size=512)


350/350 [==============================] - 1s


[0.69980263710021973, 0.0]

In [370]:
model.evaluate(x_val_negatives,y_val_negatives,batch_size=512)

30000/30000 [==============================] - 93s    


[0.68653606402079259, 1.0]

In [369]:
model.evaluate(x_test_pos,y_test_positives)

175/175 [==============================] - 0s     


[0.69980294534138265, 0.0]

In [ ]:
model.evaluate(x_test_neg,y_test_negatives,batch_size = 512)

In [ ]:
evaluate(x_val[:100],y_val[:100],model)

In [312]:
import keras

In [255]:
from keras.models import load_model
del model
model = load_model("/home/abhinav/data/second_model/the_best_model_with_trained_word_embeddings")

In [282]:
f = open("/home/abhinav/data/second_model/test_from_news")

test_news = f.read()

test_news = test_news.split("</DOC>")

x_news = tokenizer.texts_to_sequences(test_news)

x_news = pad_sequences(x_news,maxlen=MAX_SEQUENCE_LENGTH)

In [289]:
model.predict(x_news)

array([[ 0.2799862 ,  0.7200138 ],
       [ 0.53245479,  0.46754521]], dtype=float32)

In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=[X_test,y_test], 
       verbose=1, callbacks=[metrics])